In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import json
import re
import pandas as pd
from joblib import Parallel, delayed

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [3]:
def add_ad_to_car_dict(ad):
    car_url = ad
    response = requests.get(car_url)
    response.encoding = 'utf-8'
    page = BeautifulSoup(response.text, 'html.parser')
    try:
        json_data = json.loads(
            page.find('script', type="application/ld+json").string)
    except:
        json_data = None
    try:
        catalog_url = page.find(
            'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default')
        catalog_url = catalog_url.get('href')
    except:
        return
    response_catalog = requests.get(catalog_url)
    response_catalog.encoding = 'utf-8'
    page_catalog = BeautifulSoup(response_catalog.text, 'html.parser')
    try:
        json_data_catalog = json.loads(page_catalog.find(
            'script', type="application/json", id='initial-state').string)
    except:
        json_data_catalog = None
    try:
        json_data_equip = json.loads(
            page.find('script', type="application/json", id='initial-state').string)
    except:
        json_data_equip = None
    try:
        json_data['description'] = json_data['description'].replace('\n', ' ')
        json_data['description'] = re.sub('\W+', ' ', json_data['description'])
    except:
        json_data['description'] = ''
    try:
        options_dict = json_data_catalog['state']['compare']['selected'][0]['options']
        options_list = [key for key in list(
            options_dict.keys()) if options_dict[key] == 1]
    except:
        options_list = []
    car_dict.update({ad: {}})
    try:
        car_dict[ad]['car_url'] = json_data['offers']['url']
    except:
        car_dict[ad]['car_url'] = None
    try:
        car_dict[ad]['bodyType'] = json_data['bodyType']
    except:
        car_dict[ad]['bodyType'] = None
    try:
        car_dict[ad]['brand'] = json_data['brand']
    except:
        car_dict[ad]['brand'] = None
    try:
        car_dict[ad]['color'] = json_data['color']
    except:
        car_dict[ad]['color'] = None
    try:
        car_dict[ad]['complectation_dict'] = options_list
    except:
        car_dict[ad]['complectation_dict'] = None
    try:
        car_dict[ad]['description'] = json_data['description']
    except:
        car_dict[ad]['description'] = None
    try:
        car_dict[ad]['engineDisplacement'] = json_data['vehicleEngine']['engineDisplacement']
    except:
        car_dict[ad]['engineDisplacement'] = None
    try:
        car_dict[ad]['enginePower'] = json_data['vehicleEngine']['enginePower']
    except:
        car_dict[ad]['enginePower'] = None
    try:
        car_dict[ad]['equipment_dict'] = json_data_equip['card']['vehicle_info']['equipment']
    except:
        car_dict[ad]['equipment_dict'] = None
    try:
        car_dict[ad]['fuelType'] = json_data['fuelType']
    except:
        car_dict[ad]['fuelType'] = None
    try:
        car_dict[ad]['image'] = json_data['image']
    except:
        car_dict[ad]['image'] = None
    try:
        car_dict[ad]['mileage'] = page.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['mileage'] = None
    try:
        car_dict[ad]['modelDate'] = json_data['modelDate']
    except:
        car_dict[ad]['modelDate'] = None
    try:
        car_dict[ad]['model_name'] = page.find_all(
            'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
    except:
        car_dict[ad]['model_name'] = None
    try:
        car_dict[ad]['name'] = json_data['name']
    except:
        car_dict[ad]['name'] = None
    try:
        car_dict[ad]['numberOfDoors'] = json_data['numberOfDoors']
    except:
        car_dict[ad]['numberOfDoors'] = None
    try:
        car_dict[ad]['parsing_unixtime'] = int(time.time())
    except:
        car_dict[ad]['parsing_unixtime'] = None
    try:
        car_dict[ad]['price'] = json_data['offers']['price']
    except:
        car_dict[ad]['price'] = None
    try:
        car_dict[ad]['priceCurrency'] = json_data['offers']['priceCurrency']
    except:
        car_dict[ad]['priceCurrency'] = None
    try:
        car_dict[ad]['productionDate'] = json_data['productionDate']
    except:
        car_dict[ad]['productionDate'] = None
    try:
        car_dict[ad]['sell_id'] = page.find(
            'div', class_='CardHead__infoItem CardHead__id').text[2:]
    except:
        car_dict[ad]['sell_id'] = None
    try:
        car_dict[ad]['views'] = page.find(
            'div', class_='CardHead__infoItem CardHead__views').text.split()[0]
    except:
        car_dict[ad]['views'] = None
    try:
        car_dict[ad]['date_added'] = page.find(
            'div', class_='CardHead__infoItem CardHead__creationDate').text
    except:
        car_dict[ad]['date_added'] = None
    try:
        car_dict[ad]['super_gen'] = json.loads(
            page.find('div', id="sale-data-attributes").get('data-bem'))
    except:
        car_dict[ad]['super_gen'] = None
    try:
        car_dict[ad]['vehicleConfiguration'] = json_data['vehicleConfiguration']
    except:
        car_dict[ad]['vehicleConfiguration'] = None
    try:
        car_dict[ad]['vehicleTransmission'] = json_data['vehicleTransmission']
    except:
        car_dict[ad]['vehicleTransmission'] = None
    try:
        car_dict[ad]['Владельцы'] = page.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['Владельцы'] = None
    try:
        car_dict[ad]['Владение'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
    except:
        car_dict[ad]['Владение'] = None
    try:
        car_dict[ad]['ПТС'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
    except:
        car_dict[ad]['ПТС'] = None
    try:
        car_dict[ad]['Привод'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
    except:
        car_dict[ad]['Привод'] = None
    try:
        car_dict[ad]['Руль'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
    except:
        car_dict[ad]['Руль'] = None
    try:
        car_dict[ad]['Состояние'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
    except:
        car_dict[ad]['Состояние'] = None
    try:
        car_dict[ad]['Таможня'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
    except:
        car_dict[ad]['Таможня'] = None
    try:
        car_dict[ad]['region'] = page.find('div', class_='CardBreadcrumbs').find_all(
            'div', class_='CardBreadcrumbs__item')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['region'] = None

df_ads = pd.read_csv('Project_6.csv', sep=',', header=None)
all_ads = df_ads[0].tolist()
car_dict = {}
i = 90000
for ad in all_ads[90000:100000]:
    print(i)
    add_ad_to_car_dict(ad)
    i += 1
    if i % 10 == 0:
        print(f'Completed {i} of total {len(all_ads)}')
    else:
        continue

90000
90001
90002
90003
90004
90005
90006
90007
90008
90009
Completed 90010 of total 269381
90010
90011
90012
90013
90014
90015
90016
90017
90018
90019
Completed 90020 of total 269381
90020
90021
90022
90023
90024
90025
90026
90027
90028
90029
Completed 90030 of total 269381
90030
90031
90032
90033
90034
90035
90036
90037
90038
90039
Completed 90040 of total 269381
90040
90041
90042
90043
90044
90045
90046
90047
90048
90049
Completed 90050 of total 269381
90050
90051
90052
90053
90054
90055
90056
90057
90058
90059
Completed 90060 of total 269381
90060
90061
90062
90063
90064
90065
90066
90067
90068
90069
Completed 90070 of total 269381
90070
90071
90072
90073
90074
90075
90076
90077
90078
90079
Completed 90080 of total 269381
90080
90081
90082
90083
90084
90085
90086
90087
90088
90089
Completed 90090 of total 269381
90090
90091
90092
90093
90094
90095
90096
90097
90098
90099
Completed 90100 of total 269381
90100
90101
90102
90103
90104
90105
90106
90107
90108
90109
Completed 90110 of t

90891
90892
90893
90894
90895
90896
90897
90898
90899
Completed 90900 of total 269381
90900
90901
90902
90903
90904
90905
90906
90907
90908
90909
Completed 90910 of total 269381
90910
90911
90912
90913
90914
90915
90916
90917
90918
90919
Completed 90920 of total 269381
90920
90921
90922
90923
90924
90925
90926
90927
90928
90929
Completed 90930 of total 269381
90930
90931
90932
90933
90934
90935
90936
90937
90938
90939
Completed 90940 of total 269381
90940
90941
90942
90943
90944
90945
90946
90947
90948
90949
Completed 90950 of total 269381
90950
90951
90952
90953
90954
90955
90956
90957
90958
90959
Completed 90960 of total 269381
90960
90961
90962
90963
90964
90965
90966
90967
90968
90969
Completed 90970 of total 269381
90970
90971
90972
90973
90974
90975
90976
90977
90978
90979
Completed 90980 of total 269381
90980
90981
90982
90983
90984
90985
90986
90987
90988
90989
Completed 90990 of total 269381
90990
90991
90992
90993
90994
90995
90996
90997
90998
90999
Completed 91000 of total 2

91782
91783
91784
91785
91786
91787
91788
91789
Completed 91790 of total 269381
91790
91791
91792
91793
91794
91795
91796
91797
91798
91799
Completed 91800 of total 269381
91800
91801
91802
91803
91804
91805
91806
91807
91808
91809
Completed 91810 of total 269381
91810
91811
91812
91813
91814
91815
91816
91817
91818
91819
Completed 91820 of total 269381
91820
91821
91822
91823
91824
91825
91826
91827
91828
91829
Completed 91830 of total 269381
91830
91831
91832
91833
91834
91835
91836
91837
91838
91839
Completed 91840 of total 269381
91840
91841
91842
91843
91844
91845
91846
91847
91848
91849
Completed 91850 of total 269381
91850
91851
91852
91853
91854
91855
91856
91857
91858
91859
Completed 91860 of total 269381
91860
91861
91862
91863
91864
91865
91866
91867
91868
91869
Completed 91870 of total 269381
91870
91871
91872
91873
91874
91875
91876
91877
91878
91879
Completed 91880 of total 269381
91880
91881
91882
91883
91884
91885
91886
91887
91888
91889
Completed 91890 of total 269381


92673
92674
92675
92676
92677
92678
92679
Completed 92680 of total 269381
92680
92681
92682
92683
92684
92685
92686
92687
92688
92689
Completed 92690 of total 269381
92690
92691
92692
92693
92694
92695
92696
92697
92698
92699
Completed 92700 of total 269381
92700
92701
92702
92703
92704
92705
92706
92707
92708
92709
Completed 92710 of total 269381
92710
92711
92712
92713
92714
92715
92716
92717
92718
92719
Completed 92720 of total 269381
92720
92721
92722
92723
92724
92725
92726
92727
92728
92729
Completed 92730 of total 269381
92730
92731
92732
92733
92734
92735
92736
92737
92738
92739
Completed 92740 of total 269381
92740
92741
92742
92743
92744
92745
92746
92747
92748
92749
Completed 92750 of total 269381
92750
92751
92752
92753
92754
92755
92756
92757
92758
92759
Completed 92760 of total 269381
92760
92761
92762
92763
92764
92765
92766
92767
92768
92769
Completed 92770 of total 269381
92770
92771
92772
92773
92774
92775
92776
92777
92778
92779
Completed 92780 of total 269381
92780


93564
93565
93566
93567
93568
93569
Completed 93570 of total 269381
93570
93571
93572
93573
93574
93575
93576
93577
93578
93579
Completed 93580 of total 269381
93580
93581
93582
93583
93584
93585
93586
93587
93588
93589
Completed 93590 of total 269381
93590
93591
93592
93593
93594
93595
93596
93597
93598
93599
Completed 93600 of total 269381
93600
93601
93602
93603
93604
93605
93606
93607
93608
93609
Completed 93610 of total 269381
93610
93611
93612
93613
93614
93615
93616
93617
93618
93619
Completed 93620 of total 269381
93620
93621
93622
93623
93624
93625
93626
93627
93628
93629
Completed 93630 of total 269381
93630
93631
93632
93633
93634
93635
93636
93637
93638
93639
Completed 93640 of total 269381
93640
93641
93642
93643
93644
93645
93646
93647
93648
93649
Completed 93650 of total 269381
93650
93651
93652
93653
93654
93655
93656
93657
93658
93659
Completed 93660 of total 269381
93660
93661
93662
93663
93664
93665
93666
93667
93668
93669
Completed 93670 of total 269381
93670
93671


94455
94456
94457
94458
94459
Completed 94460 of total 269381
94460
94461
94462
94463
94464
94465
94466
94467
94468
94469
Completed 94470 of total 269381
94470
94471
94472
94473
94474
94475
94476
94477
94478
94479
Completed 94480 of total 269381
94480
94481
94482
94483
94484
94485
94486
94487
94488
94489
Completed 94490 of total 269381
94490
94491
94492
94493
94494
94495
94496
94497
94498
94499
Completed 94500 of total 269381
94500
94501
94502
94503
94504
94505
94506
94507
94508
94509
Completed 94510 of total 269381
94510
94511
94512
94513
94514
94515
94516
94517
94518
94519
Completed 94520 of total 269381
94520
94521
94522
94523
94524
94525
94526
94527
94528
94529
Completed 94530 of total 269381
94530
94531
94532
94533
94534
94535
94536
94537
94538
94539
Completed 94540 of total 269381
94540
94541
94542
94543
94544
94545
94546
94547
94548
94549
Completed 94550 of total 269381
94550
94551
94552
94553
94554
94555
94556
94557
94558
94559
Completed 94560 of total 269381
94560
94561
94562


95346
95347
95348
95349
Completed 95350 of total 269381
95350
95351
95352
95353
95354
95355
95356
95357
95358
95359
Completed 95360 of total 269381
95360
95361
95362
95363
95364
95365
95366
95367
95368
95369
Completed 95370 of total 269381
95370
95371
95372
95373
95374
95375
95376
95377
95378
95379
Completed 95380 of total 269381
95380
95381
95382
95383
95384
95385
95386
95387
95388
95389
Completed 95390 of total 269381
95390
95391
95392
95393
95394
95395
95396
95397
95398
95399
Completed 95400 of total 269381
95400
95401
95402
95403
95404
95405
95406
95407
95408
95409
Completed 95410 of total 269381
95410
95411
95412
95413
95414
95415
95416
95417
95418
95419
Completed 95420 of total 269381
95420
95421
95422
95423
95424
95425
95426
95427
95428
95429
Completed 95430 of total 269381
95430
95431
95432
95433
95434
95435
95436
95437
95438
95439
Completed 95440 of total 269381
95440
95441
95442
95443
95444
95445
95446
95447
95448
95449
Completed 95450 of total 269381
95450
95451
95452
95453


96237
96238
96239
Completed 96240 of total 269381
96240
96241
96242
96243
96244
96245
96246
96247
96248
96249
Completed 96250 of total 269381
96250
96251
96252
96253
96254
96255
96256
96257
96258
96259
Completed 96260 of total 269381
96260
96261
96262
96263
96264
96265
96266
96267
96268
96269
Completed 96270 of total 269381
96270
96271
96272
96273
96274
96275
96276
96277
96278
96279
Completed 96280 of total 269381
96280
96281
96282
96283
96284
96285
96286
96287
96288
96289
Completed 96290 of total 269381
96290
96291
96292
96293
96294
96295
96296
96297
96298
96299
Completed 96300 of total 269381
96300
96301
96302
96303
96304
96305
96306
96307
96308
96309
Completed 96310 of total 269381
96310
96311
96312
96313
96314
96315
96316
96317
96318
96319
Completed 96320 of total 269381
96320
96321
96322
96323
96324
96325
96326
96327
96328
96329
Completed 96330 of total 269381
96330
96331
96332
96333
96334
96335
96336
96337
96338
96339
Completed 96340 of total 269381
96340
96341
96342
96343
96344


ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))

In [4]:
df_4_c_c = pd.DataFrame.from_dict(car_dict, orient='index')
df_4_c_c.to_csv('Final_df_4_c_c.csv')

In [5]:
df_ads = pd.read_csv('Project_6.csv', sep=',', header=None)
all_ads = df_ads[0].tolist()
car_dict = {}
i = 96919
for ad in all_ads[96919:100000]:
    print(i)
    add_ad_to_car_dict(ad)
    i += 1
    if i % 10 == 0:
        print(f'Completed {i} of total {len(all_ads)}')
    else:
        continue

96919
Completed 96920 of total 269381
96920
96921
96922
96923
96924
96925
96926
96927
96928
96929
Completed 96930 of total 269381
96930
96931
96932
96933
96934
96935
96936
96937
96938
96939
Completed 96940 of total 269381
96940
96941
96942
96943
96944
96945
96946
96947
96948
96949
Completed 96950 of total 269381
96950
96951
96952
96953
96954
96955
96956
96957
96958
96959
Completed 96960 of total 269381
96960
96961
96962
96963
96964
96965
96966
96967
96968
96969
Completed 96970 of total 269381
96970
96971
96972
96973
96974
96975
96976
96977
96978
96979
Completed 96980 of total 269381
96980
96981
96982
96983
96984
96985
96986
96987
96988
96989
Completed 96990 of total 269381
96990
96991
96992
96993
96994
96995
96996
96997
96998
96999
Completed 97000 of total 269381
97000
97001
97002
97003
97004
97005
97006
97007
97008
97009
Completed 97010 of total 269381
97010
97011
97012
97013
97014
97015
97016
97017
97018
97019
Completed 97020 of total 269381
97020
97021
97022
97023
97024
97025
97026


Completed 97810 of total 269381
97810
97811
97812
97813
97814
97815
97816
97817
97818
97819
Completed 97820 of total 269381
97820
97821
97822
97823
97824
97825
97826
97827
97828
97829
Completed 97830 of total 269381
97830
97831
97832
97833
97834
97835
97836
97837
97838
97839
Completed 97840 of total 269381
97840
97841
97842
97843
97844
97845
97846
97847
97848
97849
Completed 97850 of total 269381
97850
97851
97852
97853
97854
97855
97856
97857
97858
97859
Completed 97860 of total 269381
97860
97861
97862
97863
97864
97865
97866
97867
97868
97869
Completed 97870 of total 269381
97870
97871
97872
97873
97874
97875
97876
97877
97878
97879
Completed 97880 of total 269381
97880
97881
97882
97883
97884
97885
97886
97887
97888
97889
Completed 97890 of total 269381
97890
97891
97892
97893
97894
97895
97896
97897
97898
97899
Completed 97900 of total 269381
97900
97901
97902
97903
97904
97905
97906
97907
97908
97909
Completed 97910 of total 269381
97910
97911
97912
97913
97914
97915
97916
97917


98701
98702
98703
98704
98705
98706
98707
98708
98709
Completed 98710 of total 269381
98710
98711
98712
98713
98714
98715
98716
98717
98718
98719
Completed 98720 of total 269381
98720
98721
98722
98723
98724
98725
98726
98727
98728
98729
Completed 98730 of total 269381
98730
98731
98732
98733
98734
98735
98736
98737
98738
98739
Completed 98740 of total 269381
98740
98741
98742
98743
98744
98745
98746
98747
98748
98749
Completed 98750 of total 269381
98750
98751
98752
98753
98754
98755
98756
98757
98758
98759
Completed 98760 of total 269381
98760
98761
98762
98763
98764
98765
98766
98767
98768
98769
Completed 98770 of total 269381
98770
98771
98772
98773
98774
98775
98776
98777
98778
98779
Completed 98780 of total 269381
98780
98781
98782
98783
98784
98785
98786
98787
98788
98789
Completed 98790 of total 269381
98790
98791
98792
98793
98794
98795
98796
98797
98798
98799
Completed 98800 of total 269381
98800
98801
98802
98803
98804
98805
98806
98807
98808
98809
Completed 98810 of total 2

99592
99593
99594
99595
99596
99597
99598
99599
Completed 99600 of total 269381
99600
99601
99602
99603
99604
99605
99606
99607
99608
99609
Completed 99610 of total 269381
99610
99611
99612
99613
99614
99615
99616
99617
99618
99619
Completed 99620 of total 269381
99620
99621
99622
99623
99624
99625
99626
99627
99628
99629
Completed 99630 of total 269381
99630
99631
99632
99633
99634
99635
99636
99637
99638
99639
Completed 99640 of total 269381
99640
99641
99642
99643
99644
99645
99646
99647
99648
99649
Completed 99650 of total 269381
99650
99651
99652
99653
99654
99655
99656
99657
99658
99659
Completed 99660 of total 269381
99660
99661
99662
99663
99664
99665
99666
99667
99668
99669
Completed 99670 of total 269381
99670
99671
99672
99673
99674
99675
99676
99677
99678
99679
Completed 99680 of total 269381
99680
99681
99682
99683
99684
99685
99686
99687
99688
99689
Completed 99690 of total 269381
99690
99691
99692
99693
99694
99695
99696
99697
99698
99699
Completed 99700 of total 269381


In [6]:
df_4_c_v = pd.DataFrame.from_dict(car_dict, orient='index')
df_4_c_v.to_csv('Final_df_4_c_v.csv')